In [3]:
import collections
import numpy as np
import json

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Embedding, GRU, LSTM, Bidirectional, Dropout, Activation, TimeDistributed, RepeatVector
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy


In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16692748220504959765
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12048017379565862588
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [5]:
def load_data(path):
    input_file = path
    with open(input_file, "r") as f:
        data = f.read()
    return data.split('\n')

english_sentences = load_data('/content/english.txt')
french_sentences = load_data('/content/french.txt')

In [6]:
english_sentences[:5]

['new jersey is sometimes quiet during autumn , and it is snowy in april .',
 'the united states is usually chilly during july , and it is usually freezing in november .',
 'california is usually quiet during march , and it is usually hot in june .',
 'the united states is sometimes mild during june , and it is cold in september .',
 'your least liked fruit is the grape , but my least liked is the apple .']

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')

print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

630990 English words.
228 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

608337 French words.
345 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [9]:

def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [10]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')

test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [11]:
def preprocess(x,y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 20
English vocabulary size: 200
French vocabulary size: 333


In [12]:

def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [17]:
# Ensure that both input and output data have the same number of samples
min_samples = min(tmp_x.shape[0], preproc_french_sentences.shape[0])

# Truncate both datasets to have the same number of samples
tmp_x = tmp_x[:min_samples]
preproc_french_sentences = preproc_french_sentences[:min_samples]

# Verify that input and output data have the same number of samples
assert tmp_x.shape[0] == preproc_french_sentences.shape[0], \
    f"After truncating, number of samples in x ({tmp_x.shape[0]}) and y ({preproc_french_sentences.shape[0]}) must be equal"

# Define the simple RNN model function
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    # Hyperparameters
    learning_rate = 0.005

    # Build the layers
    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model

# Create the model
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

# Train the neural network
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 101ms/step - accuracy: 0.3549 - loss: 3.5198 - val_accuracy: 0.5201 - val_loss: nan
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.5298 - loss: 1.9236 - val_accuracy: 0.5834 - val_loss: nan
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step - accuracy: 0.5842 - loss: 1.5180 - val_accuracy: 0.6177 - val_loss: nan
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.6087 - loss: 1.3761 - val_accuracy: 0.6271 - val_loss: nan
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - accuracy: 0.6207 - loss: 1.3035 - val_accuracy: 0.6488 - val_loss: nan
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.6426 - loss: 1.2176 - val_accuracy: 0.6658 - val_loss: nan
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.6589 - loss: 1.1592 - val_accuracy: 0.6863 - val_loss: nan
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - accuracy: 0.6715 - loss: 1.1078 - val_accuracy: 0.6912 - val_loss: nan
Epoch 9

In [18]:
# Print prediction(s)
print("Prediciton:")
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print('\nOriginal text:')
print(english_sentences[:1])

Prediciton:
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step
new jersey est parfois calme en l' et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [20]:
from keras.preprocessing.sequence import pad_sequences

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    # Hyperparameters
    learning_rate = 0.005

    # Build the layers
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model

# Padding the English sentences to the length of the French sentences
tmp_x = pad_sequences(preproc_english_sentences, maxlen=max_french_sequence_length, padding='post')

# Reshape the input to add a singleton dimension for the features
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

# Truncate the larger dataset to ensure the same number of samples
min_samples = min(tmp_x.shape[0], preproc_french_sentences.shape[0])
tmp_x = tmp_x[:min_samples]
preproc_french_sentences = preproc_french_sentences[:min_samples]

# Verify that input and output data have the same number of samples
assert tmp_x.shape[0] == preproc_french_sentences.shape[0], \
    f"After truncating, number of samples in x ({tmp_x.shape[0]}) and y ({preproc_french_sentences.shape[0]}) must be equal"

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(bd_rnn_model.summary())

# Train the model
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_1 (Bidirectional)      │ (None, 20, 256)             │         100,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_8 (TimeDistributed) │ (None, 20, 1024)            │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 20, 1024)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_9 (TimeDistributed) │ (None, 20, 333)             │         341,325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 705,101 (2.69 MB)

 Trainable params: 705,101 (2.69 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - accuracy: 0.3957 - loss: 3.3694 - val_accuracy: 0.5677 - val_loss: nan
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.5675 - loss: 1.6613 - val_accuracy: 0.6103 - val_loss: nan
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.6054 - loss: 1.4102 - val_accuracy: 0.6303 - val_loss: nan
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.6247 - loss: 1.3058 - val_accuracy: 0.6461 - val_loss: nan
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.6373 - loss: 1.2360 - val_accuracy: 0.6568 - val_loss: nan
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.6480 - loss: 1.1840 - val_accuracy: 0.6637 - val_loss: nan
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.6563 - loss: 1.1344 - val_accuracy: 0.6692 - val_loss: nan
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.6597 - loss: 1.1096 - val_accuracy: 0.6743 - val_loss: nan
Ep

In [21]:
# Print prediction(s)
print("Prediciton:")
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print('\nOriginal text:')
print(english_sentences[:1])

Prediciton:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step
new jersey est parfois calme en l' et il est neigeux en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [23]:
from keras.preprocessing.sequence import pad_sequences

def bidirectional_embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    # Hyperparameters
    learning_rate = 0.005

    # Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_shape=input_shape[1:]))
    model.add(Bidirectional(GRU(256, return_sequences=True)))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model

# Padding the English sentences to the length of the French sentences
tmp_x = pad_sequences(preproc_english_sentences, maxlen=max_french_sequence_length, padding='post')

# Reshape the input to add a singleton dimension for the features
tmp_x = tmp_x.reshape((-1, max_french_sequence_length))

# Truncate the larger dataset to ensure the same number of samples
min_samples = min(tmp_x.shape[0], preproc_french_sentences.shape[0])
tmp_x = tmp_x[:min_samples]
preproc_french_sentences = preproc_french_sentences[:min_samples]

# Build the model
embed_rnn_model = bidirectional_embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(embed_rnn_model.summary())

# Train the neural network
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 20, 256)             │          51,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 20, 512)             │         789,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_12                  │ (None, 20, 1024)            │         525,312 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 20, 1024)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_13                  │ (None, 20, 333)             │         341,325 │
│ (TimeDistributed)                    │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,707,341 (6.51 MB)

 Trainable params: 1,707,341 (6.51 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 159ms/step - accuracy: 0.4212 - loss: 3.3533 - val_accuracy: 0.6849 - val_loss: nan
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 130ms/step - accuracy: 0.7062 - loss: 1.1025 - val_accuracy: 0.7893 - val_loss: nan
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step - accuracy: 0.8029 - loss: 0.6566 - val_accuracy: 0.8587 - val_loss: nan
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 130ms/step - accuracy: 0.8589 - loss: 0.4488 - val_accuracy: 0.8935 - val_loss: nan
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 129ms/step - accuracy: 0.8917 - loss: 0.3391 - val_accuracy: 0.9162 - val_loss: nan
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 129ms/step - accuracy: 0.9154 - loss: 0.2645 - val_accuracy: 0.9290 - val_loss: nan
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step - accuracy: 0.9307 - loss: 0.2184 - val_accuracy: 0.9393 - val_loss: nan
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 128ms/step - accuracy: 0.9426 - loss: 0.1806 - val_accuracy: 0.9456 - val_loss:

In [24]:
# Print prediction(s)
print("Prediciton:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print('\nOriginal text:')
print(english_sentences[:1])

Prediciton:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 486ms/step
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [26]:
# Save the model with a .keras extension
embed_rnn_model.save('english_to_french_model.keras')

# Serialize English Tokenizer to JSON
with open('english_tokenizer.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(english_tokenizer.to_json(), ensure_ascii=False))

# Serialize French Tokenizer to JSON
with open('french_tokenizer.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(french_tokenizer.to_json(), ensure_ascii=False))

# Save max lengths
max_french_sequence_length_json = max_french_sequence_length
with open('sequence_length.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(max_french_sequence_length_json, ensure_ascii=False))
